In [1]:
import pandas as pd
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [18]:
data = pd.read_csv('accident.csv')
data

,STATE,STATENAME,YEAR,ROUTENAME,RUR_URBNAME,FUNC_SYSNAME,RD_OWNERNAME,TYP_INTNAME
0,1,Alabama,2021,Interstate,Urban,Interstate,State Highway Agency,Not an Intersection
1,1,Alabama,2021,Interstate,Urban,Interstate,State Highway Agency,Not an Intersection
2,1,Alabama,2021,Interstate,Urban,Interstate,State Highway Agency,Not an Intersection
3,1,Alabama,2021,Interstate,Urban,Interstate,State Highway Agency,Not an Intersection
4,1,Alabama,2021,Interstate,Rural,Interstate,State Highway Agency,Not an Intersection
...,...,...,...,...,...,...,...,...
39503,56,Wyoming,2021,County Road,Rural,Minor Collector,County Highway Agency,Not an Intersection
39504,56,Wyoming,2021,State Highway,Rural,Major Collector,State Highway Agency,Not an Intersection
39505,56,Wyoming,2021,Interstate,Rural,Interstate,State Highway Agency,Not an Intersection
39506,56,Wyoming,2021,U.S. Highway,Urban,Principal Arterial - Other,State Highway Agency,Not an Intersection


In [24]:
states = pd.read_csv('50+states.csv')

In [19]:
g_state = data.groupby(['STATENAME'])
g_state = g_state.count()
g_state = g_state.reset_index()


g_int = data.groupby(['STATENAME', 'TYP_INTNAME'])
g_int = g_int.count()
g_int = g_int.reset_index()

In [21]:
state_total = g_state[['STATENAME', 'TYP_INTNAME']]
state_total = state_total.rename(columns={'TYP_INTNAME':'TOTAL'})
state_total

,STATENAME,TOTAL
0,Alabama,885
1,Alaska,59
2,Arizona,1063
3,Arkansas,631
4,California,3983
5,Colorado,638
6,Connecticut,283
7,Delaware,132
8,District of Columbia,40
9,Florida,3451


In [22]:
joined = pd.merge(state_total, g_int, on = ['STATENAME'])
joined = joined.rename(columns={'STATE':'COUNT'})
joined = joined[['STATENAME', 'TOTAL', 'TYP_INTNAME', 'COUNT']]
joined['RATE'] = round((joined['COUNT'] / joined['TOTAL']) * 100, 2)
joined

,STATENAME,TOTAL,TYP_INTNAME,COUNT,RATE
0,Alabama,885,Four-Way Intersection,91,10.28
1,Alabama,885,Not an Intersection,727,82.15
2,Alabama,885,T-Intersection,57,6.44
3,Alabama,885,Y-Intersection,10,1.13
4,Alaska,59,Four-Way Intersection,6,10.17
...,...,...,...,...,...
283,Wisconsin,572,Y-Intersection,3,0.52
284,Wyoming,102,Four-Way Intersection,1,0.98
285,Wyoming,102,Not an Intersection,95,93.14
286,Wyoming,102,Roundabout,1,0.98


In [26]:
combined = pd.merge(joined, states[['State', 'Abbr']], left_on=['STATENAME'], right_on=['State'])
combined = combined[['STATENAME', 'TOTAL', 'TYP_INTNAME', 'COUNT', 'RATE', 'Abbr']]
combined

,STATENAME,TOTAL,TYP_INTNAME,COUNT,RATE,Abbr
0,Alabama,885,Four-Way Intersection,91,10.28,AL
1,Alabama,885,Not an Intersection,727,82.15,AL
2,Alabama,885,T-Intersection,57,6.44,AL
3,Alabama,885,Y-Intersection,10,1.13,AL
4,Alaska,59,Four-Way Intersection,6,10.17,AK
...,...,...,...,...,...,...
280,Wisconsin,572,Y-Intersection,3,0.52,WI
281,Wyoming,102,Four-Way Intersection,1,0.98,WY
282,Wyoming,102,Not an Intersection,95,93.14,WY
283,Wyoming,102,Roundabout,1,0.98,WY


In [27]:
## fourway intersecton
fway = combined[combined['TYP_INTNAME'] == 'Four-Way Intersection']

## not an intersection
n_int = combined[combined['TYP_INTNAME'] == 'Not an Intersection']

## t-intersection
t_int = combined[combined['TYP_INTNAME'] == 'T-Intersection']

In [8]:
## csv files

# fway.to_csv('fway.csv')
# n_int.to_csv('n_int.csv')
# t_int.to_csv('t_int.csv')

In [33]:
import plotly.express as px
fig = px.choropleth(fway,
                    locations='Abbr', 
                    locationmode="USA-states", 
                    scope="usa",
                    color='RATE',
                    color_continuous_scale="Viridis_r",
                    title = 'Rate of Accidents at a Four-Way Intersection in 2021 by States')
fig.show()

In [35]:
fig = px.choropleth(t_int,
                    locations='Abbr', 
                    locationmode="USA-states", 
                    scope="usa",
                    color='RATE',
                    color_continuous_scale="Viridis_r",
                    title = 'Rate of Accidents at a T-Intersection in 2021 by States')
fig.show()